In [ ]:
"""
Inplace conversion of .wav files to .opus format and updating corresponding JSON files.

This script performs the following tasks:
1. Recursively searches for .wav files in the specified input directory and its subdirectories.
2. Converts each .wav file to .opus format using the ffmpeg command-line tool.
3. Updates the corresponding JSON file by replacing the .wav extension with .opus in the 'audio_path' field.
4. Removes the original .wav file after successful conversion.
5. Utilizes multithreading to process the files concurrently for improved performance.

The conversion process uses the following ffmpeg settings:
- Audio codec: libopus
- Bitrate: 32K (defined in FFMPEG_BITRATE constant)
- Application: voip (defined in FFMPEG_APPLICATION constant)
- Frame duration: 40 (defined in FFMPEG_FRAME_DURATION constant)
- Compression level: Randomly selected between 1 and 9 for each file

Constants:
- FFMPEG_BITRATE: The bitrate used for the ffmpeg conversion.
- FFMPEG_APPLICATION: The application setting used for the ffmpeg conversion.
- FFMPEG_FRAME_DURATION: The frame duration used for the ffmpeg conversion.
- MAX_WORKER_THREADS: The maximum number of worker threads used for concurrent processing.

Functions:
- compression_level(): Returns a random integer between 1 and 9 for the compression level.
- convert_to_opus(input_path: Path): Converts a .wav file to .opus format using ffmpeg.
- update_json(json_path: Path): Updates the 'audio_path' field in the corresponding JSON file.
- process_file(wav_path: Path): Processes a single .wav file by converting it to .opus and updating the JSON file.
- process_files(input_dir: Path): Processes all .wav files in the specified input directory and its subdirectories.

Usage:
1. Set the `input_dir` variable to the desired input directory path.
2. Run the script.

Note:
- The script requires the ffmpeg command-line tool to be installed and accessible from the system's PATH.
- The script assumes that each .wav file has a corresponding JSON file with the same name (except for the extension).
- The script modifies the JSON files in-place and removes the original .wav files after successful conversion.
- The script utilizes multithreading with a maximum number of worker threads defined by the MAX_WORKER_THREADS constant.
- The estimated time left is calculated based on the average processing time per file and the number of files left.
- Error handling is implemented for subprocess calls, JSON file loading/writing, and file I/O operations.
- The `pathlib` module is used for file path handling and manipulation.

Example:
input_dir = Path("X:/vits")
process_files(input_dir)
"""

import os
import subprocess
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import random
from pathlib import Path

# Constants
FFMPEG_BITRATE = "32K"
FFMPEG_APPLICATION = "voip"
FFMPEG_FRAME_DURATION = "40"
MAX_WORKER_THREADS = 6

def compression_level():
    return random.randint(1, 9)

def convert_to_opus(input_path: Path):
    output_path = input_path.with_suffix(".opus")
    if not output_path.exists():
        print(f"Converting {input_path} to {output_path}")
        cmd = f"ffmpeg -y -i \"{input_path}\" -c:a libopus -b:a {FFMPEG_BITRATE} -application {FFMPEG_APPLICATION} -frame_duration {FFMPEG_FRAME_DURATION} -compression_level {compression_level()} \"{output_path}\""
        try:
            subprocess.run(cmd, shell=True, check=True)
            input_path.unlink()  # Remove the original .wav file after successful conversion
        except subprocess.CalledProcessError as e:
            print(f"Error converting {input_path}: {e}")
    else:
        print(f"{output_path} already exists. Skipping conversion.")

def update_json(json_path: Path):
    if json_path.exists():
        try:
            with json_path.open("r", encoding="utf-8") as json_file:
                data = json.load(json_file)
                data["audio_path"] = data["audio_path"].replace(".wav", ".opus")
            
            with json_path.open("w", encoding="utf-8", newline="\n") as json_file:
                json.dump(data, json_file, ensure_ascii=False, indent=4)
        except json.JSONDecodeError as e:
            print(f"Error loading JSON file {json_path}: {e}")
        except IOError as e:
            print(f"Error writing JSON file {json_path}: {e}")

def process_file(wav_path: Path):
    convert_to_opus(wav_path)
    json_path = wav_path.with_suffix(".json")
    update_json(json_path)

def process_files(input_dir: Path):
    wav_files = list(input_dir.rglob("*.wav"))
    total_files = len(wav_files)
    print(f"Total .wav files to convert: {total_files}")

    start_time = time.time()
    completed_files = 0

    with ThreadPoolExecutor(max_workers=MAX_WORKER_THREADS) as executor:
        future_to_wav = {executor.submit(process_file, wav_path): wav_path for wav_path in wav_files}

        for future in as_completed(future_to_wav):
            try:
                future.result()
                completed_files += 1
                elapsed_time = time.time() - start_time
                files_left = total_files - completed_files
                avg_time_per_file = elapsed_time / completed_files
                est_time_left = avg_time_per_file * files_left / 60  # Calculate estimated time left in minutes

                print(f"Completed {completed_files}/{total_files}. Estimated time left: {est_time_left:.1f} minutes.\n")
            except Exception as exc:
                wav_path = future_to_wav[future]
                print(f"{wav_path} generated an exception: {exc}")

    print("Inplace conversion process completed.")


input_dir = Path("X:/vits")
process_files(input_dir)
